# SARAH Toolbox
## Collection of models to be run from a docker image
Launch virtual docker image: 
[![Binder](http://mybinder.org/badge.svg)](http://mybinder.org/repo/restrepo/radinert-DTFDTSDM-Toolbox)

[Jupyter home](./) (Files, New -> Terminal, etc)
[![Home](http://www.incredimail.com/images/nav%20bar/home-icon.png)](./)

### Highly recommended:
[Run from a terminal](../terminals/1):
[![T](http://mysitemyway.com/_shared/images/icons/full_color/terminal.png)](../terminals/1)
## Implemented models
Each model is to be run in a specific virtual machine. Follow the binder button in each github repo
* `SARAH/Models/SSDM/` (for test purposes)
* `SARAH/Models/SM/HighScale` (For RGE running: ./butler SM/HighScale) [Repo](https://github.com/restrepo/SM-Toolbox/)
* `SARAH/Models/SimplifiedDM/DFDM` (This image)
* `SARAH/Models/SimplifiedDM/SDFDM` (This image)

## Instructions to compile the model
In `SPHENO` and `micromegas`

See possible analysis based on the models in  [./tests](./tests) folder.

Below we define the model to be compiled:

In [1]:
MODEL=radinertDTFDTSDM

To better control in the outputs, it is recomended that the commands be executed [from a terminal](../terminals/1). Only possible errors are to be shown below

In [2]:
./compile_spheno_directly.sh $MODEL > /dev/null

make a work dir

In [3]:
mkdir -p test_compilation
cd test_compilation

In [4]:
LHAINPUT=../SPHENO/$MODEL/Input_Files/LesHouches.in.$MODEL
if [ -f ../SARAH/Models/$MODEL/LesHouches.in.$MODEL ]; then
   LHAINPUT=../SARAH/Models/$MODEL/LesHouches.in.$MODEL
fi

Run SPheno and generate LHA output file

In [5]:
cp $LHAINPUT . #only a number of limited charactes accepted as input
../SPHENO/bin/SPheno$MODEL LesHouches.in.$MODEL

 Calculating branching ratios and decay widths
 Calculating low energy constraints
 Writing output files
 Finished!


In [6]:
pwd

/home/restrepo/prog/toolbox/toolbox/test_compilation


Compile `micromegas` model

In [7]:
cd ../micromegas
make > /dev/null
make > /dev/null
cd $MODEL
make main=CalcOmega.cpp > /dev/null

ar: creating work_aux.a
ar: creating aLib.a


### Include Indirect Detection

In [8]:
micromegas_with_DDetection=CalcOmega_with_DDetection_MOv4.2.cpp 
IDmsg="======== Indirect Detection ========"
if [ -f "$micromegas_with_DDetection" ]  && [ ! "$(grep "$IDmsg" "$micromegas_with_DDetection" )" ]; then
    tmpfile=$(mktemp /tmp/CalcOmega_UP.XXXXXX)
    grep -B 1000 'fclose(channels);' "$micromegas_with_DDetection" | grep -v 'fclose(channels);' > "$tmpfile"
    echo "
{
  double Emin=1,/* Energy cut  in GeV   */  sigmaV;
  double vcs_gz,vcs_gg;
  char txt[100];
  double SpA[NZ],SpE[NZ],SpP[NZ];
  double FluxA[NZ],FluxE[NZ],FluxP[NZ];
  double * SpNe=NULL,*SpNm=NULL,*SpNl=NULL;
  double Etest=Mcdm/2;

  printf(\"\n$IDmsg\n\"); 
  sigmaV=calcSpectrum(1+2+4,SpA,SpE,SpP,SpNe,SpNm,SpNl ,&err);

}" >> "$tmpfile"

  grep -A 1000 'fclose(channels);' "$micromegas_with_DDetection" >> "$tmpfile"
  mv "$tmpfile" "$micromegas_with_DDetection"
else 
  echo "$IDmsg"
  echo "already included"
fi

Run micromegas

In [9]:
make main="$micromegas_with_DDetection" > /dev/null

In [10]:
cd ../../test_compilation
../micromegas/$MODEL/$(basename "$micromegas_with_DDetection" .cpp) SPheno.spc.$MODEL 


Masses of odd sector Particles:
~Nv1 : MNv1  =     0.0 || ~Nv2 : MNv2  =     0.0 || ~Nv3 : MNv3  =     0.0 
~ci1 : Mci1  =     0.0 || ~ci2 : Mci2  =     0.0 || ~etI : MetI  =    98.5 
~XP1 : MXP1  =   100.0 || ~XO1 : MXO1  =   101.5 || ~XP2 : MXP2  =   560.0 
~XO2 : MXO2  =   560.0 || 
PROCESS: ~Nv1,~Nv1->AllEven,1*x{h,g,A,Z,Wp,Wm,nu1,nu2,nu3,d1,D1,d2,D2,d3,D3,u1,U1,u2,U2,u3,U3,e1,E1,e2,E2,e3,E3
PROCESS: Z->2*x
PROCESS: Wp->2*x
PROCESS: u1->2*x
PROCESS: ~Nv1,~Nv2->AllEven,1*x{h,g,A,Z,Wp,Wm,nu1,nu2,nu3,d1,D1,d2,D2,d3,D3,u1,U1,u2,U2,u3,U3,e1,E1,e2,E2,e3,E3
PROCESS: ~Nv1,~Nv3->AllEven,1*x{h,g,A,Z,Wp,Wm,nu1,nu2,nu3,d1,D1,d2,D2,d3,D3,u1,U1,u2,U2,u3,U3,e1,E1,e2,E2,e3,E3
PROCESS: ~Nv1,~ci1->AllEven,1*x{h,g,A,Z,Wp,Wm,nu1,nu2,nu3,d1,D1,d2,D2,d3,D3,u1,U1,u2,U2,u3,U3,e1,E1,e2,E2,e3,E3
PROCESS: ~Nv1,~ci2->AllEven,1*x{h,g,A,Z,Wp,Wm,nu1,nu2,nu3,d1,D1,d2,D2,d3,D3,u1,U1,u2,U2,u3,U3,e1,E1,e2,E2,e3,E3
PROCESS: ~Nv2,~Nv2->AllEven,1*x{h,g,A,Z,Wp,Wm,nu1,nu2,nu3,d1,D1,d2,D2,d3,D3,u1,U1,u2,U2,u3,U3,e1,E1,e2

## Go now to: 
* [main ipyrhon file](./tests/radinert-DTFDTSDM.ipynb) → radinertDTFDTSDM: Singlet Doublet Triplet Dark matter